In [2]:
import cv2
import glob
import os
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import random

In [10]:
#Contrast enhancement function
# converting to LAB color space
def contrast_enhancement (img):
    lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)

    # Applying CLAHE to L-channel
    # feel free to try different values for the limit and grid size:
    clahe = cv2.createCLAHE(clipLimit=20.0, tileGridSize=(8,8))
    cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L-channel with the a and b channel
    limg = cv2.merge((cl,a,b))

    # Converting image from LAB Color model to BGR color spcae
    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    return enhanced_img

In [3]:
def extract_hand(image):
    if image is None:
        print('Error: Could not load image')
        return None
# Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Remove the shadow using the RGB color space
    b, g, r = cv2.split(image)
    bg_diff = cv2.absdiff(b, g)
    br_diff = cv2.absdiff(b, r)
    shadow_mask = cv2.bitwise_and(bg_diff, br_diff)
    _, shadow_mask = cv2.threshold(shadow_mask, 35, 255, cv2.THRESH_BINARY)
    shadow_mask = cv2.morphologyEx(shadow_mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)))
    shadow_mask = cv2.merge([shadow_mask, shadow_mask, shadow_mask])
    img_no_shadow = cv2.subtract(image, shadow_mask)

    # Convert the image to grayscale
    gray_no_shadow = cv2.cvtColor(img_no_shadow, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to convert the image to a binary image
    thresh = cv2.adaptiveThreshold(gray_no_shadow, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Remove noise from the binary image
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # Remove small contours and fill holes in the hand region
    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    hand_mask = np.zeros_like(closed)
    for cnt in contours:
        if cv2.contourArea(cnt) > 10000:
            cv2.drawContours(hand_mask, [cnt], 0, (255, 255, 255), -1)

    # Remove any remaining artifacts from the shadow
    shadow_mask = cv2.bitwise_not(hand_mask)
    shadow_mask = cv2.morphologyEx(shadow_mask, cv2.MORPH_OPEN, kernel)
    shadow_mask = cv2.morphologyEx(shadow_mask, cv2.MORPH_CLOSE, kernel)
    shadow_mask = cv2.bitwise_not(shadow_mask)

    # Apply the mask to remove the shadow
    hand = cv2.bitwise_and(closed, shadow_mask)

    # Invert the image to get the hand region as white and the background as black
    hand = cv2.bitwise_not(hand)

    # Return the hand image
    return hand

In [5]:
# Load the dataset of images
# Each image is a grayscale image of size 64x128

# dataset_path = 'Dataset_0-5-001/men/0/'
# image_paths = glob.glob(os.path.join(dataset_path, '*.JPG'))
# image_paths.append(glob.glob(os.path.join('Dataset_0-5-001/men/1/', '*.JPG')))

dataset_path_men = 'Dataset_0-5-001/men/'
image_paths_men = []

for root, dirs, files in os.walk(dataset_path_men):
    for file in files:
        if file.endswith('.JPG'):
            image_paths_men.append(os.path.join(root, file))

dataset_path_women = 'Dataset_0-5-001/women/'
image_paths_women = []

for root, dirs, files in os.walk(dataset_path_women):
    for file in files:
        if file.endswith('.JPG'):
            image_paths_women.append(os.path.join(root, file))

image_paths = image_paths_men + image_paths_women

print(len(image_paths_men))
print(len(image_paths_women))
print(len(image_paths))

# Define the HOG parameters
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (3, 3)
block_norm = 'L2-Hys'

# Extract the HOG features from each image
hog_features = []
labels = []
for image_path in image_paths:
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    
    enhanced_image = extract_hand(image)
    if enhanced_image is not None:
        hog_feature = hog(enhanced_image, orientations=orientations,
                        pixels_per_cell=pixels_per_cell,
                        cells_per_block=cells_per_block,
                        block_norm=block_norm)
        hog_features.append(hog_feature)
        label = os.path.basename(image_path).split('.')[0][0]
        labels.append(int(label))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(hog_features, labels, test_size=0.2, random_state=42)

# Apply PCA to the data 
pca = PCA(n_components=10)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

# Train the SVM classifier on the PCA-transformed data
svm = LinearSVC()
svm.fit(X_train_pca, y_train)

# Evaluate the classifier on the PCA-transformed testing data
y_pred = svm.predict(X_test_pca)
print('y_test',y_test)
print('y_pred',y_pred)
accuracy = accuracy_score(y_test, y_pred)

print('Accuracy:', accuracy)

1053
773
1826


error: OpenCV(4.6.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0xf2302844::Set<1,-1,-1>,struct cv::impl::A0xf2302844::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1
